In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize # Import the minimize function

# Diversified portfolio of six stocks
tickers = ['MSFT', 'JNJ', 'AAPL', 'GOOG', 'BHP', 'TSLA']  # Microsoft, Johnson & Johnson, Apple, Google, BHP Billiton, Tesla

# Download historical data for the past 6 months
data = yf.download(tickers=tickers, period='6mo')['Adj Close']

# Keep only the last 5 months of data
data = data.iloc[-150:] 

# Calculate daily returns and create a DataFrame
returns = data.pct_change().dropna()  
returns_df = pd.DataFrame(returns)

# Drop any rows with NaN values (essential for accurate covariance)
returns_df.dropna(inplace=True)

# Calculate the variance-covariance matrix
C = returns_df.cov()

print(C)

[*********************100%%**********************]  6 of 6 completed

Ticker      AAPL       BHP          GOOG           JNJ      MSFT      TSLA
Ticker                                                                    
AAPL    0.000236  0.000019  7.412244e-05  4.709814e-06  0.000070  0.000194
BHP     0.000019  0.000220  4.480892e-05  1.144973e-05  0.000043  0.000134
GOOG    0.000074  0.000045  3.055401e-04  7.886806e-07  0.000115  0.000003
JNJ     0.000005  0.000011  7.886806e-07  8.338207e-05  0.000004 -0.000003
MSFT    0.000070  0.000043  1.152599e-04  3.558667e-06  0.000144  0.000064
TSLA    0.000194  0.000134  2.801136e-06 -3.225369e-06  0.000064  0.001248


In [15]:
# --- METHOD 1 IMPLEMENTATION ---

w = np.ones(6)/6   # Initial guess for weight matrix (equal weights)
cov_matrix_method1 = np.array(C)  # Convert covariance matrix to NumPy array for Method 1

def obj_function_method1(w, cov_matrix_method1):
    P = cov_matrix_method1 @ w
    MCR = P/np.sqrt(w.T @ cov_matrix_method1 @ w)
    f = 0.0
    for i in range(len(w)):
        for j in range(len(w)):
            f += (w[i] * MCR[i] - w[j] * MCR[j]) ** 2
    return f

def objective_method1(w):
    return obj_function_method1(w, cov_matrix_method1)

# Constraints
constraints_method1 = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum of weights = 1
    {'type': 'ineq', 'fun': lambda w: w}             # Weights are non-negative
)

# Bounds
bounds_1 = [(0, 1) for _ in range(cov_matrix_method1.shape[1])]

# Optimize
result_method1 = minimize(objective_method1, w, method='SLSQP',constraints=constraints_method1, bounds=bounds_1, options={'disp': True})

# Optimal weights
optimal_weights_method1 = result_method1.x

print("Objective function value at optimal weights:", result_method1.fun)
print("Success:", result_method1.success)
print("Message:", result_method1.message)
print("Optimal weights (Method 1):", optimal_weights_method1)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.111575279028734e-05
            Iterations: 5
            Function evaluations: 35
            Gradient evaluations: 5
Objective function value at optimal weights: 2.111575279028734e-05
Success: True
Message: Optimization terminated successfully
Optimal weights (Method 1): [0.1778087  0.18159745 0.18612129 0.18099713 0.18389077 0.08958466]


In [12]:
# --- METHOD 2 IMPLEMENTATION ---

w = np.ones(6) / 6  # Initial guess for weight matrix (equal weights)
cov_matrix_method2 = np.array(C)  # Convert covariance matrix to NumPy array for Method 2

def obj_function_method2(w, cov_matrix_method2):
    return np.sqrt(w.T @ cov_matrix_method2 @ w)

def objective_method2(w):
    return obj_function_method2(w, cov_matrix_method2)

# Constraints
constraints_method2 = (
    {'type': 'ineq', 'fun': lambda w: np.sum(np.log(w)) + 2},  # Logarithmic constraint
    {'type': 'ineq', 'fun': lambda w: w}             # Weights are non-negative
)

# Bounds
bounds_2 = [(0, 1) for _ in range(cov_matrix_method2.shape[1])]

# Optimize
result_method2 = minimize(objective_method2, w, method='SLSQP',constraints=constraints_method2, bounds=bounds_2, options={'disp': True})

# Optimal weights
optimal_weights_method2 = result_method2.x

# Normalize the weights (Method 2)
y = np.array(optimal_weights_method2)
w_method2 = y / np.sum(y)  # Normalized weights for Method 2

print("\nOptimal weights (Method 2):", optimal_weights_method2)  # Unnormalized weights
print("Objective function value at optimal weights:", result_method2.fun)
print("Success:", result_method2.success)
print("Message:", result_method2.message)
print("Normalized weights (Method 2):", w_method2)  # Normalized weights

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.03856473666691075
            Iterations: 25
            Function evaluations: 176
            Gradient evaluations: 25

Optimal weights (Method 2): [0.73698449 0.84602872 0.68739118 1.         0.86984847 0.36301115]
Objective function value at optimal weights: 0.03856473666691075
Success: True
Message: Optimization terminated successfully
Normalized weights (Method 2): [0.16365563 0.18787011 0.15264288 0.22206115 0.19315955 0.08061068]


In [13]:
# --- Calculating Expected Return and Variance using method 2 ---

# Let's use Method 2 weights 
weights_method2 = w_method2  # Normalized weights from Method 2

# Calculate portfolio expected return
expected_return_method2 = np.sum(weights_method2 * returns_df.mean())

# Calculate portfolio variance (Method 2)
portfolio_variance_method2 = weights_method2.T @ C @ weights_method2

# --- Calculate Market Capitalization Weights ---

# Get market cap data from Yahoo Finance (replace with actual data)
market_cap_data_for_MSFT = yf.Ticker("MSFT").info['marketCap']
market_cap_data_for_JNJ = yf.Ticker("JNJ").info['marketCap']
market_cap_data_for_AAPL = yf.Ticker("AAPL").info['marketCap']
market_cap_data_for_GOOG = yf.Ticker("GOOG").info['marketCap']
market_cap_data_for_BHP = yf.Ticker("BHP").info['marketCap']
market_cap_data_for_TSLA = yf.Ticker("TSLA").info['marketCap']

market_cap = pd.DataFrame(index=tickers)
market_cap['MarketCap'] = [market_cap_data_for_MSFT,
                            market_cap_data_for_JNJ,
                            market_cap_data_for_AAPL,
                            market_cap_data_for_GOOG,
                            market_cap_data_for_BHP,
                            market_cap_data_for_TSLA]

market_cap_weights = market_cap['MarketCap'] / market_cap['MarketCap'].sum()

# Calculate portfolio variance (Market Cap)
portfolio_variance_market_cap = market_cap_weights.T @ C @ market_cap_weights

# --- Output ---

print("\nExpected return (Method 2):", expected_return_method2)
print("Portfolio variance (Method 2):", portfolio_variance_method2)
print("Portfolio variance (Market Cap):", portfolio_variance_market_cap)

# Compare the variances
print("\nVariance difference (Risk Parity - Market Cap):", portfolio_variance_method2 - portfolio_variance_market_cap)


Expected return (Method 2): 0.0008481454142075849
Portfolio variance (Method 2): 7.333746982013192e-05
Portfolio variance (Market Cap): 0.00011990058810776094

Variance difference (Risk Parity - Market Cap): -4.656311828762902e-05
